In [ ]:
import os
import requests

!pip install ultralytics
from ultralytics import YOLO


In [ ]:
# Configuración
output_file = "alt_texts.txt"  # Archivo donde se guardarán los textos alternativos
model_path = "yolov8n.pt"  # Ruta al modelo YOLO (puedes usar un modelo preentrenado como yolov8n.pt)
base_url = "https://iiif.ohc.cu/iiif/3/{}/full/max/0/default.jpg"  # URL base para las imágenes
file_list = "list.txt"  # Archivo con los nombres de las imágenes

# Cargar el modelo
model = YOLO(model_path)


In [ ]:

# Crear o vaciar el archivo de salida
with open(output_file, "w") as f:
    f.write("")  # Vaciar el archivo

# Leer nombres de archivo desde list.txt
with open(file_list, "r") as file:
    image_names = [line.strip() for line in file.readlines()]



In [ ]:

# Procesar las imágenes
for image_name in image_names:
    url = base_url.format(image_name)

    try:


        # Descargar la imagen desde la URL con un tiempo límite de 10 segundos
        try:
            response = requests.get(url, stream=True, timeout=10)
            if response.status_code == 200:
                # Guardar temporalmente la imagen descargada
                temp_image_path = f"{image_name}.jpg"
                with open(temp_image_path, "wb") as temp_image:
                    for chunk in response.iter_content(1024):
                        temp_image.write(chunk)
            else:
                print(f"No se pudo descargar la imagen para {image_name} (Código HTTP {response.status_code})")
                continue
        except requests.exceptions.Timeout:
            print(f"Tiempo de espera excedido para {image_name}")
            continue
        except Exception as e:
            print(f"Error al descargar {image_name}: {e}")
            continue



        if response.status_code == 200:
            # Guardar temporalmente la imagen descargada
            temp_image_path = f"{image_name}.jpg"
            with open(temp_image_path, "wb") as temp_image:
                for chunk in response.iter_content(1024):
                    temp_image.write(chunk)

            # Realizar detección con YOLO
            results = model(temp_image_path)

            # Extraer información de detección
            alt_texts = []
            for result in results:
                for box in result.boxes.data:
                    cls = int(box[5])  # Clase detectada
                    confidence = float(box[4])  # Confianza de la predicción
                    label = model.names[cls]  # Nombre de la clase
                    if confidence > 0.6:
                        alt_texts.append(f"{label}")

            # Crear el texto alternativo
            alt_text = f"{', '.join(alt_texts)}"

            # Guardar en el archivo de salida
            with open(output_file, "a") as f:
                f.write(alt_text + "\n")

            print(f"Procesada: {image_name} -> {alt_text}")

            # Eliminar la imagen temporal
            os.remove(temp_image_path)
        else:
            print(f"No se pudo descargar la imagen para {image_name} (Código HTTP {response.status_code})")

    except Exception as e:
        print(f"Error al procesar {image_name}: {e}")